In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## NEURALNETWORKSANDDEEPLEARNING/TUNINGNEURALNETWORKS/NEURALNETWORKSANDDEEPLEARNING TUNINGNEURALNETWORKS 1 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [3]:
#=================================================-
#### Slide 3: Loading packages  ####

# Helper packages.
import os
import pickle
import pandas as pd
import numpy as np
# Scikit-learn packages.
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

# TensorFlow and supporting packages.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from keras_tuner.tuners import RandomSearch

from tensorflow.keras.layers import Dropout

In [4]:
import neptune.new as neptune

ImportError: cannot import name 'ParamSpec' from 'typing_extensions' (/usr/local/Caskroom/mambaforge/base/envs/tf/lib/python3.9/site-packages/typing_extensions.py)

In [ ]:
from neptune.new.integrations.tensorflow_keras import NeptuneCallback

In [5]:
%pip install --upgrade type_extensions
# %conda install -yc conda-forge typing_extensions

Note: you may need to restart the kernel to use updated packages.


In [ ]:
#=================================================-
#### Slide 6: Setting up project parameters for Neptune  ####

# Initialize your Neptune client.
run = neptune.init(project='USER_NAME/PROJECT_NAME', #<- track your project
                   api_token = 'API_TOKEN')          #<- set your API token
             




In [11]:
#=================================================-
#### Slide 7: Directory settings  ####

from pathlib import Path 
home_dir = Path(".").resolve()
main_dir = home_dir.parent
print(main_dir)
data_dir = str(main_dir) + "/data"
print(data_dir)




/Users/matskarz/Skillsoft-Content/IntrotoNeural_Networks
/Users/matskarz/Skillsoft-Content/IntrotoNeural_Networks/data


In [12]:
#=================================================-
#### Slide 8: Load the data  ####

credit_card = pd.read_csv(str(data_dir) + '/credit_card_data.csv')
print(credit_card.head())




   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1      20000    2          2         1   24      2      2     -1     -1   
1   2     120000    2          2         2   26     -1      2      0      0   
2   3      90000    2          2         2   34      0      0      0      0   
3   4      50000    2          2         1   37      0      0      0      0   
4   5      50000    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...          0          0          0         0       689         0   
1  ...       3272       3455       3261         0      1000      1000   
2  ...      14331      14948      15549      1518      1500      1000   
3  ...      28314      28959      29547      2000      2019      1200   
4  ...      20940      19146      19131      2000     36681     10000   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default_payment_next_month  
0         0         0   

In [13]:
#=================================================-
#### Slide 10: Data prep: convenience function - cont'd  ####

def data_prep(df):
    
    # Fill missing values with mean 
    df = df.fillna(df.mean()['BILL_AMT1'])

    # Drop an unnecessary identifier column.
    df = df.drop('ID',axis = 1)

    # Convert 'sex' into dummy variables.
    sex = pd.get_dummies(df['SEX'], prefix = 'sex', drop_first = True)
    # Convert 'education' into dummy variables.
    education = pd.get_dummies(df['EDUCATION'], prefix = 'education', drop_first = True)
    # Convert 'marriage' into dummy variables.
    marriage = pd.get_dummies(df['MARRIAGE'], prefix = 'marriage', drop_first = True)

    # Drop `sex`, `education`, `marriage` from the data.
    df.drop(['SEX', 'EDUCATION', 'MARRIAGE'], axis = 1, inplace = True)

    # Concatenate `sex`, `education`, `marriage` dummies to our dataset.
    df = pd.concat([df, sex, education, marriage], axis=1)
    
    # Separate predictors from data.
    X = df.drop(['default_payment_next_month'], axis=1)

    # Separate target from data.
    y = df['default_payment_next_month']

    # Set the seed to 1.
    np.random.seed(1)
    
    # Split data into train, test, and validation set, use a 70 - 15 - 15 split.
    # First split data into train-test with 70% for train and 30% for test.
    X_train, X_test, y_train, y_test = train_test_split(X.values,
                                                        y,
                                                        test_size = .3,
                                                        random_state = 1)
    
    # Then split the test data into two halves: test and validation. 
    X_test, X_val, y_test, y_val = train_test_split(X_test,
                                                    y_test,
                                                    test_size = .5,
                                                    random_state = 1)
                                                    
    print("Train shape:", X_train.shape, "Test shape:", X_test.shape, "Val shape:", X_val.shape)
    
    # Transforms features by scaling each feature to a given range.
    # The default is the range between 0 and 1.
    min_max_scaler = preprocessing.MinMaxScaler()
    X_train_scaled = min_max_scaler.fit_transform(X_train)
    X_test_scaled = min_max_scaler.transform(X_test)
    X_val_scaled = min_max_scaler.transform(X_val)
    
    return X_train_scaled, X_test_scaled, X_val_scaled, y_train, y_test, y_val




In [14]:
#=================================================-
#### Slide 11: Data prep  ####

X_train_scaled, X_test_scaled, X_val_scaled, y_train, y_test, y_val = data_prep(credit_card)





Train shape: (21000, 30) Test shape: (4500, 30) Val shape: (4500, 30)


In [15]:
#=================================================-
#### Slide 12: Define metrics  ####

METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc')
]




2022-11-18 12:47:45.430825: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
#=================================================-
#### Slide 13: Define model function  ####

def create_model(lr = .75):
  # Let's set the seed so that we can reproduce the results.
  tf.random.set_seed(1)
  opt = Adam(learning_rate = lr) # <- set optimizer

  model = Sequential([
          Dense(32, activation='relu', 
                  input_dim=X_train_scaled.shape[1]),#<- set input and 1st hidden layer
          Dense(32, activation='relu'),              #<- set 2nd hidden layer
          Dense(1, activation='sigmoid')             #<- set output layer

  ])

  model.compile(optimizer = opt,            #<- set optimizer
                loss='binary_crossentropy', #<- set loss function to binary_crossentropy
                metrics= METRICS)           #<- set performance metric
  return model




In [ ]:
#=================================================-
#### Slide 14: Launching Neptune: create callback  ####

neptune_cbk = NeptuneCallback(run=run)
callbacks = [neptune_cbk]

# Create and compile the model.
model = create_model()




In [ ]:
#=================================================-
#### Slide 15: Fitting the model with Neptune callback  ####

# Fit the model with NeptuneCallback
model_default = model.fit(X_train_scaled, y_train,
                          validation_data = (X_val_scaled, y_val),
                          epochs = 25,
                          verbose = 0,          #<- silence the epoch output in console (optional)
                          callbacks = callbacks)#<- add callbacks




In [ ]:
#=================================================-
#### Slide 21: Evaluate the model  ####

model.evaluate(X_test_scaled, y_test)
run.stop()


#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################


In [ ]:
#######################################################
#######################################################
############    COPYRIGHT - DATA SOCIETY   ############
#######################################################
#######################################################

## NEURALNETWORKSANDDEEPLEARNING/TUNINGNEURALNETWORKS/NEURALNETWORKSANDDEEPLEARNING TUNINGNEURALNETWORKS 2 ##

## NOTE: To run individual pieces of code, select the line of code and
##       press ctrl + enter for PCs or command + enter for Macs




In [17]:
#=================================================-
#### Slide 4: Define the tuner (cont'd)  ####

def tune_model(hp):

    # Number of 1st hidden layer neurons.
    units = hp.Int('units',        #<- Number of neurons
                   min_value = 8,  #<- min value
                   max_value = 64, #<- max value
                   step = 8)
    
    # Tuning activation function for 1st hidden layer.
    activation = hp.Choice('activation', #<- Activation function
                             [             #<- Types of activation functions
                                 'relu',
                                 'tanh',
                                 'sigmoid'
                             ])
    
    dropout_1 = hp.Float('dropout_1',
                         min_value = 0.0,
                         max_value = 0.5,
                         default = 0.25,
                         step = 0.05)
    
    # Tuning optimizer.
    optimizer = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    
    # Tuning learning rate
    lr = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])

    model = keras.Sequential([
        keras.layers.Dense(
            units=units,
            activation=activation,
            input_dim=X_train_scaled.shape[1],
        ),
        keras.layers.Dense(units=units, activation=activation),
        Dropout(rate=dropout_1),
        keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    # Compile model.
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = METRICS)
    
    return model
MAX_TRIALS = 10
EXECUTIONS_PER_TRIAL = 5
tuner = RandomSearch(
    tune_model,
    objective = 'val_accuracy',
    max_trials = MAX_TRIALS,
    executions_per_trial = EXECUTIONS_PER_TRIAL,
    directory = 'final_tuned_model',
    project_name = 'final_tuned_model',
    seed = 1
)

In [18]:
#=================================================-
#### Slide 5: View search space summary  ####

tuner.search_space_summary()




Search space summary
Default search space size: 5
units (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 64, 'step': 8, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.05, 'sampling': None}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'sgd', 'rmsprop'], 'ordered': False}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [19]:
#=================================================-
#### Slide 6: Fit the model  ####

tuner.search(x=X_train_scaled,
             y=y_train,
             verbose=0,
             epochs=25,
             validation_data=(X_val_scaled, y_val))




2022-11-18 12:48:40.677378: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


INFO:tensorflow:Oracle triggered exit


In [20]:
#=================================================-
#### Slide 7: View the optimal parameters  ####

optimal_params = tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values
optimal_params




{'units': 56,
 'activation': 'tanh',
 'dropout_1': 0.25,
 'optimizer': 'adam',
 'learning_rate': 0.001}

In [21]:
#=================================================-
#### Slide 8: Define and compile optimized model  ####

def create_optimized_model(units, activation, dropout_1, optimizer, learning_rate, dropout_seed = 1):

    # Set up model.
    model = Sequential()
    
    model.add(Dense(units,
                    input_dim = X_train_scaled.shape[1],
                    activation = activation))
    model.add(Dense(units,
                    activation = activation))
    
    if dropout_1 is not None:
        model.add(Dropout(rate = dropout_1, seed = dropout_seed))
        model.add(Dense(1, activation = 'sigmoid'))
    # Compile model.
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer, metrics = METRICS)
    return model




In [ ]:
#=================================================-
#### Slide 9: Setup Neptune run for optimized model  ####

# Initialize your Neptune client.
run = neptune.init(project='USER_NAME/PROJECT_NAME', #<- track your project
                   api_token = 'API_TOKEN')          #<- set your API token
             
neptune_cbk = NeptuneCallback(run=run)
callbacks = [neptune_cbk]
# Create and compile the optimized model.
tb_model = create_optimized_model(**optimal_params)




In [ ]:
#=================================================-
#### Slide 10: Fit the optimized model  ####

tb_model.fit(X_train_scaled,
             y_train,
             validation_data = (X_val_scaled, y_val),
             epochs = 25,
             verbose = 0,          #<- silence the epoch output in console (optional)
             callbacks = callbacks)#<- add callbacks





In [ ]:
#=================================================-
#### Slide 11: Evaluate optimized model on test data  ####

tb_model.evaluate(X_test_scaled, y_test)
run.stop()




In [ ]:
#=================================================-
#### Slide 25: Exercise   ####




#######################################################
####  CONGRATULATIONS ON COMPLETING THIS MODULE!   ####
#######################################################
